## Imports

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 29.8 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np

import os
import mediapipe as mp
import tensorflow as tf
from matplotlib import pyplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Mediapipe functions

####2

In [ ]:
def marks(results):
    marks = [0 for i in range(126)]
    num_hands = len(results.multi_handedness)
    for i in range(num_hands):
        isLeft = results.multi_handedness[i].classification[0].label == "Left"
        
        landmarks = results.multi_hand_world_landmarks[i]
        if isLeft:
            c = 0
            #count = 0
            for l in landmarks.landmark:
              #if count % 2 == 0:
                marks[c] = l.x
                marks[c+1] = l.y
                marks[c+2] = l.z
                c += 3
              #count += 1
        else:
            c = 63
            #count = 0
            for l in landmarks.landmark:
              #if count % 2 == 0:
                marks[c] = l.x
                marks[c+1] = l.y
                marks[c+2] = l.z
                c += 3
              #count += 1
    return np.array(marks)

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def mediapipe_detection2(image):
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw(image, results):
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
          mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                    mp_drawing_styles.get_default_hand_landmarks_style(),
                                    mp_drawing_styles.get_default_hand_connections_style())            

#### test

In [ ]:
sequence = []
cap = cv2.VideoCapture("/content/drive/MyDrive/data/words/Познакомиться/video5215591976824481994.mp4")
with mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue
        #image = cv2.flip(image, 1)
        image, results = mediapipe_detection2(image)  
        draw(image, results)
        
        if cv2.waitKey(5) & 0xFF == 27:
            break
            
        if results.multi_hand_world_landmarks:
            keypoints = marks(results)
            
        else:
          keypoints = np.zeros(126)
        sequence.append(keypoints)
        if len(sequence) == 20:
            out = model.predict(np.array(sequence).reshape(1, 20, 126))
            print(out[0])
            sequence = []

cap.release()

In [ ]:
cv2.destroyAllWindows()

## Make dirs for npy data

In [ ]:
data_path = '/content/drive/MyDrive/data/words'

In [ ]:
DATA_PATH = '/content/drive/MyDrive/code/data126_npy2/слова85'
no_sequences = 85

In [ ]:
#actions = np.array(os.listdir(data_path))
actions = np.array(['Привет', 'дела', 'До свидания', 'зовут', 'Как', 'любить', 'Я', 'Здравствуйте', 'Можно', 'Вы', 'Познакомиться', 'где',
       'школа', 'учиться'])

In [ ]:
actions

array(['Привет', 'дела', 'До свидания', 'зовут', 'Как', 'любить', 'Я',
       'Здравствуйте', 'Можно', 'Вы', 'Познакомиться', 'где', 'школа',
       'учиться'], dtype='<U13')

In [ ]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## Record

In [ ]:
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    for action in actions:
        for sequence in range(85):
            path = os.path.join(data_path, action)
            video = os.path.join(path, os.listdir(path)[sequence])
            print(video, sequence)
            cap = cv2.VideoCapture(video)
            for frame_num in range(30):
                success, image = cap.read()
                if not success:
                    continue
                #image = cv2.resize(image, (1280, 720))
                #image = cv2.flip(image, 1)
                image, results = mediapipe_detection2(image)  
                #draw(image, results)

                '''if frame_num == 0: 
                    cv2.putText(image, 'Начинается запись', (120,200), 
                               cv2.FONT_HERSHEY_COMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Запись фреймов для {} видео {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Запись фреймов для {} видео {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))'''
                    
                if results.multi_hand_landmarks:
                    keypoints = marks(results)
                else:
                  keypoints = np.zeros(126)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

## Label map

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

{'Привет': 0,
 'дела': 1,
 'До свидания': 2,
 'зовут': 3,
 'Как': 4,
 'любить': 5,
 'Я': 6,
 'Здравствуйте': 7,
 'Можно': 8,
 'Вы': 9,
 'Познакомиться': 10,
 'где': 11,
 'школа': 12,
 'учиться': 13}

## Dataset

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
sequences, labels = [], []

In [ ]:
for action in actions:
    print(action)
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(10, 30):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(str(frame_num))))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

Привет
дела
До свидания
зовут
Как
любить
Я
Здравствуйте
Можно
Вы
Познакомиться
где
школа
учиться


In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

(1190, 20, 126)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y.shape

(1190, 14)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

## Train

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import *
from tensorflow.keras.layers import *

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

#### lstm

In [ ]:
X, Y = np.array(sequences), to_categorical(labels).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=34, stratify=Y)

model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(20,126)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=False, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, Y_train, epochs=300)

#### 1. conv 1d

In [ ]:
window_size = 20
numFilters1 = 100
kernalSize1 = 3
kernalSize2 = 2
numNueronsFCL2 = 160
dropout = 0.5
num_classes = 14
num_data_parameters = 126

In [ ]:
model_m = Sequential()
model_m.add(Reshape((window_size, num_data_parameters), input_shape=(window_size, num_data_parameters)))
model_m.add(Conv1D(numFilters1, kernalSize1, activation='relu', input_shape=(window_size, num_data_parameters)))
model_m.add(MaxPooling1D(2))
model_m.add(Conv1D(numNueronsFCL2, kernalSize2, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(dropout))
model_m.add(Dense(num_classes, activation='softmax'))

model_m.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='/content/logs', histogram_freq=1)

model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 128
EPOCHS = 500

history = model_m.fit(X_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=[tensorboard_callback],
                      validation_split=0.2,
                      verbose=1)

#### 2. conv 1d

In [ ]:
n_timesteps, n_features, n_outputs = X.shape[1], X.shape[2], y.shape[1]

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='/content/logs', histogram_freq=1)

model.summary()

In [ ]:
verbose, epochs, batch_size = 1, 500, 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_split=0.2, callbacks=[tensorboard_callback])

In [ ]:
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(accuracy)

8/8 [==============================] - 0s 7ms/step - loss: 0.0427 - accuracy: 0.9958
0.9957982897758484


## Save model

In [ ]:
model.save("model")

## Load model

In [ ]:
model = tf.keras.models.load_model('model2.h5')

## TFlite convert

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

96528

## Evaluate

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, mean_squared_error, recall_score, precision_score

In [ ]:
yhat = model.predict(X_test)

8/8 [==============================] - 0s 3ms/step


In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

0.9327731092436975

In [ ]:
recall_score(ytrue, yhat, average="micro")

0.9327731092436975

In [ ]:
precision_score(ytrue, yhat, average="micro")

0.9327731092436975

In [ ]:
mean_squared_error(ytrue, yhat)

0.7394957983193278